In [2]:
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
)
from datasets import load_dataset, load_metric
import torch


dataset=load_dataset('multi_news')

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.1 MB/s 
     |████████████████████████████████| 120 kB 60.5 MB/s 
     |████████████████████████████████| 6.6 MB 42.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 6.8 MB/s 
     |████████████████████████████████| 212 kB 52.2 MB/s 
     |████████████████████████████████| 115 kB 52.8 MB/s 
     |████████████████████████████████| 127 kB 54.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
TOKENIZER = AutoTokenizer.from_pretrained('ratishsp/Centrum-multinews')
MODEL = LEDForConditionalGeneration.from_pretrained('ratishsp/Centrum-multinews')#.to("cuda")
MODEL.gradient_checkpointing_enable()
PAD_TOKEN_ID = TOKENIZER.pad_token_id
DOCSEP_TOKEN_ID = TOKENIZER.convert_tokens_to_ids("<doc-sep>")

In [4]:
def process_document(documents):
    input_ids_all=[]
    for data in documents:
        all_docs = data.split("|||||")[:-1]
        for i, doc in enumerate(all_docs):
            doc = doc.replace("\n", " ")
            doc = " ".join(doc.split())
            all_docs[i] = doc

        #### concat with global attention on doc-sep
        input_ids = []
        for doc in all_docs:
            input_ids.extend(
                TOKENIZER.encode(
                    doc,
                    #truncation=True,
                    #max_length=4096 // len(all_docs),
                )[1:-1]
            )
            input_ids.append(DOCSEP_TOKEN_ID)
        input_ids = (
            [TOKENIZER.bos_token_id]
            + input_ids
            + [TOKENIZER.eos_token_id]
        )
        input_ids_all.append(torch.tensor(input_ids))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids_all, batch_first=True, padding_value=PAD_TOKEN_ID
    )
    return input_ids


def batch_process(batch):
    input_ids=process_document(batch['document'])
    # get the input ids and attention masks together
    global_attention_mask = torch.zeros_like(input_ids).to(input_ids.device)
    # put global attention on <s> token

    global_attention_mask[:, 0] = 1
    global_attention_mask[input_ids == DOCSEP_TOKEN_ID] = 1
    generated_ids = MODEL.generate(
        input_ids=input_ids,
        global_attention_mask=global_attention_mask,
        use_cache=True,
        max_length=1024,
        num_beams=5,
    )
    generated_str = TOKENIZER.batch_decode(
            generated_ids.tolist(), skip_special_tokens=True
        )
    result={}
    result['generated_summaries'] = generated_str
    result['gt_summaries']=batch['summary']
    return result

In [5]:
#import random
#data_idx = random.choices(range(len(dataset['test'])),k=10)
#dataset_small = dataset['test'].select(data_idx)
dataset_small = dataset['test'].select([1,2,3,4,5,6,7,8,9,10])
from datetime import datetime
start_time = datetime.now()
result_small = dataset_small.map(batch_process, batched=True, batch_size=2)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

  0%|          | 0/5 [00:00<?, ?ba/s]

Duration: 0:05:35.825927


In [6]:
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu sacrebleu
import evaluate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=685c648d9668e8ea79cfa2b080f7f1310a269fcfcd164760a000244ebbd8559f
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 8.4 MB/s 


In [11]:

result_small['generated_summaries']

['– Richard Metzger of Boing Boing thinks Facebook has taken down a photo of two men kissing in a British soap opera. "Hey Facebook what’s SO wrong with a pic of two men kissing?" he asks on Boing Boing. "I used this particular photo because I considered it to be quite mild (no groping, no tongues). The photos I had considered using before I chose that one are much more racy. Oh the irony!" Metzger says he used the photo to illustrate his post about the "John Snow Kiss-In." He says he used it because he considered it "quite mild (no groping, no tongues). The photos I had considered using before I chose that one are much more racy. Oh the irony!" He adds that he made the event private after it was over because "there were starting to be trolls posting abusive nonsense on it." Metzger says he\'s not sure why Facebook took the photo down. "It has been erroneously reported in the media that our own Richard Metzger (who lives in Los Angeles) organized the London \'Kiss-In\' event, which is 

In [8]:
rouge = load_metric("rouge")
score=rouge.compute(predictions=result_small["generated_summaries"], references=result_small["gt_summaries"])
print(score['rouge1'].mid)
print(score['rouge2'].mid)
print(score['rougeL'].mid)

Score(precision=0.4693381094884187, recall=0.45172807523465897, fmeasure=0.4498249289713244)
Score(precision=0.1917272283116468, recall=0.17161715415258838, fmeasure=0.17808877419259725)
Score(precision=0.26498685068142136, recall=0.241087184306775, fmeasure=0.24805839616344047)


In [9]:
rouge = evaluate.load("rouge")
score = rouge.compute(predictions=result_small["generated_summaries"], references=result_small["gt_summaries"],rouge_types=['rouge1', 'rouge2','rouge3','rougeL'],use_aggregator=False,use_stemmer=True)
#print(score['rouge1'].mid)
#print(score['rouge2'].mid)
#print(score['rouge3'].mid)
#print(score['rougeL'].mid)
score  

{'rouge1': [0.47754137115839246,
  0.46004842615012104,
  0.4012251148545176,
  0.3707165109034267,
  0.4914004914004914,
  0.4409937888198758,
  0.5607064017660045,
  0.2305295950155763,
  0.4512534818941505,
  0.7870036101083032],
 'rouge2': [0.13776722090261284,
  0.1265206812652068,
  0.055299539170506916,
  0.090625,
  0.1382716049382716,
  0.18125,
  0.2039911308203991,
  0.031347962382445145,
  0.15126050420168066,
  0.7127272727272728],
 'rouge3': [0.0477326968973747,
  0.024449877750611245,
  0.009244992295839754,
  0.028213166144200632,
  0.05459057071960297,
  0.12578616352201258,
  0.1024498886414254,
  0.0,
  0.07323943661971831,
  0.6666666666666666],
 'rougeL': [0.1749408983451537,
  0.1937046004842615,
  0.16232771822358347,
  0.15576323987538943,
  0.21621621621621623,
  0.25465838509316774,
  0.26490066225165565,
  0.11838006230529595,
  0.24512534818941503,
  0.7509025270758122]}

In [10]:
chrf = evaluate.load("chrf")
resultsc1 = chrf.compute(predictions=[result_small["generated_summaries"][0]], references=[result_small["gt_summaries"][0]],beta=1,word_order=0)
print(resultsc1)

{'score': 39.06253173728513, 'char_order': 6, 'word_order': 0, 'beta': 1}


In [ ]:
result_small['generated_summaries']

'"I grabbed a hold of her arm, and I wrapped it around my neck and grabbed her body and bear hugged her and dragged her over," Perez said. "2-0-2. It\'s code four. I got her off the bridge," Perez could be heard saying afterwards over the police radio. Perez said the girl didn\'t seem happy after the rescue. "She was sad and just started breaking down even more," he said. By the time paramedics arrived, Perez\'s job was done -- but not forgotten. Now when he looks at the bridge, fear goes through his mind, he said. She had told him she only had an aunt -- no other family -- and wanted to live with her. For now, the 12-year-old is waiting in protective police custody..'

In [12]:
dataset_small['document']

['\n \n \n \n UPDATE: 4/19/2001 Read Richard Metzger: How I, a married, middle-aged man, became an accidental spokesperson for gay rights overnight on Boing Boing \n \n It’s time to clarify a few details about the controversial “Hey Facebook what’s SO wrong with a pic of two men kissing?” story, as it now beginning to be reported in the mainstream media, and not always correctly. \n \n First of all, with regards to the picture: \n \n The photo which was used to illustrate my first post about the John Snow Kiss-In is a promotional still from the British soap opera “Eastenders.” It features one of the main characters from the show (Christian Clarke, played by the actor John Partridge- left) and someone else who I don’t know. I am not a regular viewer so I can’t say if the man on the right is an extra or an actual character. \n \n This picture has itself caused scandal in the UK, as it was a gay kiss that was broadcast before the watershed, and as such led to a number of complaints to the

In [ ]:
len(dataset_small['document'])

10